# IF3170 Artificial Intelligence | Tugas Besar 2

Group Number: 31 - Kolmogorov-Arnold

Group Members:
- Roby Pratama Sitepu (13121142)
- Muhammad Fadli Alfarizi (13121140)
- Muhammad Arviano Yuono (13621034)
- Hafizh Renanto Akhmad (13621060)

## Import Libraries

In [6]:
from src import CustomTransformer


def reset_import():
    for module_name in list(sys.modules.keys()):
        if 'src' in module_name:
            del sys.modules[module_name]
            # print(f"removed module: {module_name}")


def check_for_completeness(data):
    print(data.shape)
    count = data.isna().sum().sum()

    if count > 0:
        raise Exception(f'\n{count} missing values found')


def inspect(df, features=None):
    for feature in features if features is not None else df.columns:
        print(f"feature = '{feature}'")
        print(f"count unique values = {df[feature].nunique()}")
        print(f"unique values: {df[feature].unique()}")
        print(f"missing values: {df[feature].isna().sum()}\n\n")


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import sys

reset_import()

## Import Dataset

In [7]:
train_file_id = ['1sLJLWnim9A34A8hpQwZ9PxVhMJPOrN9d', '1GzheGwc1mG6z_M_kYUOMMPkE2QQrj0eW',
                 '1BunbVg1gBwI9v8h_6FxUMXJcuJDmnmMR', '1gkIqZAd5Aza8F8-2MzMO6t53gLEUYquj',
                 '1p38XDB0XStCbYKYVBi-Pmi2BunD6FY6J']

test_file_id = ['1TdeZx1bhHL8340emZhz8rW80016kdum3', '1Hg1_obL7mpwjN9gN0g1Me4jMlo8LGmVn',
                '1G8spITfMGnVry6XJGz3ONg3N-CFXltrI', '1NFcFBcHVj3LguL9BRC39HuAZTif2ws_8',
                '1tfmJu2HoeAePv7qKvIjAGETaqWkkhdbj']

X = pd.read_csv(f"https://drive.google.com/uc?id={train_file_id[0]}")
y = pd.read_csv('https://drive.google.com/uc?id=1Dt9cKiT1y1cysxyJGlcnnEkms8ibSjCq')[['attack_cat', 'id']]
for file_id in train_file_id[1:]:
    X = pd.merge(X, pd.read_csv(f"https://drive.google.com/uc?id={file_id}"), on='id')

shuffled = pd.merge(X, y, on='id').sample(frac=1, random_state=42).reset_index(drop=True)

X = shuffled.drop(columns=['attack_cat'])
y = shuffled[['attack_cat', 'id']]

X.set_index('id', inplace=True)
y.set_index('id', inplace=True)

categorical_features = ['proto', 'state', 'service', 'is_sm_ips_ports', 'is_ftp_login']
numerical_features = [col for col in X.columns if
                      col not in categorical_features and col != 'attack_cat' and col != 'id']

# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [ ]:
# Write your code here

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
for feature in categorical_features:
    unique_values_X_train = X_train[feature].dropna().nunique()
    unique_values_X_val = X_val[feature].dropna().nunique()

    if unique_values_X_train != unique_values_X_val:
        print(f"Feature '{feature}' has different unique values: "
              f"{unique_values_X_train} in X_train and {unique_values_X_val} in X_val.")
        vals = list(set(X_train[feature].unique().tolist()) - set(X_val[feature].unique().tolist()))
        for val in vals:
            print(f"{val} in column '{feature}' {(X_train[feature] == val).sum()} times")

            y_values = y_train[X_train[feature] == val]
            print(f"corresponding y values for {val} in column '{feature}': {y_values['attack_cat'].tolist()}")


Feature 'proto' has different unique values: 133 in X_train and 132 in X_val.
rtp in column 'proto' 1 times
corresponding y values for rtp in column 'proto': ['Normal']
Feature 'state' has different unique values: 8 in X_train and 7 in X_val.
PAR in column 'state' 1 times
corresponding y values for PAR in column 'state': ['Normal']
URN in column 'state' 1 times
corresponding y values for URN in column 'state': ['Normal']


In [16]:
print(len(numerical_features))

for col in numerical_features:
    print(f"Unique values in '{col}': {X_train[col].nunique()}")

36
Unique values in 'ct_state_ttl': 5
Unique values in 'ct_flw_http_mthd': 11
Unique values in 'ct_ftp_cmd': 4
Unique values in 'ct_srv_src': 52
Unique values in 'ct_srv_dst': 52
Unique values in 'ct_dst_ltm': 49
Unique values in 'ct_src_ltm': 50
Unique values in 'ct_src_dport_ltm': 47
Unique values in 'ct_dst_sport_ltm': 32
Unique values in 'ct_dst_src_ltm': 54
Unique values in 'dur': 51381
Unique values in 'sbytes': 5617
Unique values in 'dbytes': 5187
Unique values in 'sttl': 11
Unique values in 'dttl': 5
Unique values in 'sloss': 335
Unique values in 'dloss': 299
Unique values in 'sload': 55845
Unique values in 'dload': 52894
Unique values in 'spkts': 409
Unique values in 'dpkts': 377
Unique values in 'swin': 9
Unique values in 'dwin': 6
Unique values in 'stcpb': 50930
Unique values in 'dtcpb': 50747
Unique values in 'smean': 1328
Unique values in 'dmean': 1280
Unique values in 'trans_depth': 10
Unique values in 'response_body_len': 1877
Unique values in 'sjit': 52735
Unique values

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:
reset_import()

from src.compose import CustomTransformer

preprocessor = CustomTransformer(
    transformers=[
        (SimpleImputer(strategy='mean'), numerical_features),
        (SimpleImputer(strategy='most_frequent'), categorical_features),
    ]
)

X_train_completed = preprocessor.fit_transform(X_train)
X_val_completed = preprocessor.transform(X_val)

check_for_completeness(X_train_completed)
check_for_completeness(X_val_completed)

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
reset_import()

from src.preprocessing.outlier.strategy import BoundOutlierHandlerStrategy

preprocessor = CustomTransformer(
    transformers=[
        (BoundOutlierHandlerStrategy(), numerical_features)
    ]
)

X_train_outliers_removed = preprocessor.fit_transform(X_train_completed)
X_val_outliers_removed = preprocessor.transform(X_val_completed)

check_for_completeness(X_train_outliers_removed)
check_for_completeness(X_val_outliers_removed)

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
reset_import()

from src.preprocessing.duplicate import DuplicateHandler
from src.preprocessing.duplicate import DropDuplicateHandlerStrategy

duplicate_handler = DuplicateHandler(strategy=DropDuplicateHandlerStrategy())

X_train_duplicates_removed = duplicate_handler.transform(X_train_outliers_removed)
X_val_duplicates_removed = duplicate_handler.transform(X_val_outliers_removed)

y_train_duplicates_removed = y_train.loc[y_train.index.isin(X_train_duplicates_removed.index)]
y_val_duplicates_removed = y_val.loc[y_val.index.isin(X_val_duplicates_removed.index)]

check_for_completeness(X_train_duplicates_removed)
check_for_completeness(X_val_duplicates_removed)

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:

   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.

   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
reset_import()

from src.preprocessing.encoder import FeatureEncoder, OneHotFeatureEncoderStrategy

feature_encoder = FeatureEncoder(strategy=OneHotFeatureEncoderStrategy())

X_train_encoded = feature_encoder.fit_transform(X_train_duplicates_removed)
X_val_encoded = feature_encoder.transform(X_val_duplicates_removed)

check_for_completeness(X_train_encoded)
check_for_completeness(X_val_encoded)

In [ ]:
reset_import()

from src.preprocessing.selection import FeatureSelectionHandler
from src.preprocessing.selection.strategy import RandomForestFeatureSelectionHandlerStrategy
from src.compose import PipelineProperty

pl_property = PipelineProperty({'numerical_features': numerical_features, 'categorical_features': categorical_features})

feature_selection_handler = FeatureSelectionHandler(
    strategy=RandomForestFeatureSelectionHandlerStrategy(),
)

X_train_selected = feature_selection_handler.fit_transform(X_train_encoded, y_train_duplicates_removed['attack_cat'])
X_val_selected = feature_selection_handler.transform(X_val_encoded)

check_for_completeness(X_train_selected)
check_for_completeness(X_val_selected)

selected_numerical_features = [feature for feature in X_train_selected.columns if feature in numerical_features]
selected_categorical_features = [feature for feature in X_train_selected.columns if
                                 feature not in selected_numerical_features]

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
reset_import()

from src.compose import CustomTransformer
from src.preprocessing.scale import ZScoreScaleHandlerStrategy

preprocessing = CustomTransformer(
    transformers=[
        (ZScoreScaleHandlerStrategy(), selected_numerical_features),
    ]
)

X_train_scaled = preprocessing.fit_transform(X_train_selected)
X_val_scaled = preprocessing.transform(X_val_selected)

check_for_completeness(X_train_scaled)
check_for_completeness(X_val_scaled)

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
# already encoded before feature selection (random forest)

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
# Write your code here

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("missing", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [ ]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [12]:
reset_import()

from abc import ABC, abstractmethod
from src.preprocessing import TransformerStrategy


class Model(ABC):
    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass


class Proxy(TransformerStrategy):
    def __init__(self, model: Model):
        self._model = model

    def fit(self, X, y):
        self._model.fit(X, y)

    def transform(self, X):
        return self._model.predict(X)

    def fit_transform(self, X, y):
        self.fit(X, y)

        return self.transform(X)


class SKLearnLabelEncoderProxy():
    def __init__(self):
        self._encoders = {}

    @abstractmethod
    def fit(self, X, y=None):
        for feature in X.columns:
            encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                                     unknown_value=-1).fit(X[feature].values.reshape(-1, 1))
            self._encoders[feature] = encoder

    @abstractmethod
    def transform(self, X):
        X_transformed = X.copy()
        for feature in X_transformed.columns:
            X_transformed[feature] = self._encoders[feature].transform(X_transformed[feature].values.reshape(-1, 1))

        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)

        return self.transform(X)

component: missing
component: outliers
component: feature selection
component: feature scaling
component: encoder
component: classifier
component: missing
component: outliers
component: feature selection
component: feature scaling
component: encoder
component: classifier
(122738, 33)
component: missing
component: outliers
component: feature selection
component: feature scaling
component: encoder
component: classifier

Model Accuracy: 0.7799365055224987
F1 Score (Macro): 0.5527688651378088


# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

In [10]:
from src.compose import CustomPipeline, PipelineProperty, CustomTransformer, PipelineComponent
from src.preprocessing.outlier import OutlierHandler, BoundOutlierHandlerStrategy
from src.preprocessing.selection import FeatureSelectionHandler, RandomForestFeatureSelectionHandlerStrategy
from src.preprocessing.missing import MissingValueHandler
from src.preprocessing.duplicate import DuplicateHandler, DropDuplicateHandlerStrategy
from src.preprocessing.scale import ScaleHandler, ZScoreScaleHandlerStrategy
from src.compose.io import ClassifierTransformerStrategy, LabelEncoderIOTransformerStrategy

## A. KNN

In [61]:
reset_import()

from sklearn.neighbors import KNeighborsClassifier

import warnings

warnings.filterwarnings('ignore')

reset_import()

from src.compose import CustomPipeline

pipeline = CustomPipeline(args=[
    ('missing', MissingValueHandler(strategy=CustomTransformer(transformers=[
        (SimpleImputer(strategy='mean'), numerical_features),
        (SimpleImputer(strategy='most_frequent'), categorical_features)
    ]))),
    ('duplicate', DuplicateHandler(strategy=DropDuplicateHandlerStrategy())),
    ('outliers', OutlierHandler(strategy=CustomTransformer(transformers=[
        (BoundOutlierHandlerStrategy(), numerical_features)
    ]))),
    ('feature selection', FeatureSelectionHandler(strategy=CustomTransformer(transformers=[
        (RandomForestFeatureSelectionHandlerStrategy(), numerical_features),
    ]))),
    ('feature scaling', ScaleHandler(strategy=ZScoreScaleHandlerStrategy()), 'numerical_features'),
    ('encoder', PipelineComponent(strategy=None).set_io_strategy(
        LabelEncoderIOTransformerStrategy(strategy=CustomTransformer(transformers=[
            (SKLearnLabelEncoderProxy(), categorical_features),
        ])),
    )),
    ('classifier', PipelineComponent(strategy=None).set_io_strategy(
        ClassifierTransformerStrategy(strategy=Proxy(KNeighborsClassifier(n_neighbors=3))))),
], pl_property=PipelineProperty(
    {'numerical_features': numerical_features, 'categorical_features': categorical_features}), verbose=True,
    only_once=['duplicate'])

X_train_transformed, _ = pipeline.fit_transform(X_train, y_train)

check_for_completeness(X_train_transformed)

_, y_pred = pipeline.transform(X_val)

accuracy = accuracy_score(y_val, y_pred)
print("\nModel Accuracy:", accuracy)

f1_macro = f1_score(y_val, y_pred, average='macro')
print(f"F1 Score (Macro): {f1_macro}")

fit: missing
fit: duplicate
fit: outliers
fit: feature selection
fit: feature scaling
fit: encoder
fit: classifier
transform: missing
transform: outliers
transform: feature selection
transform: feature scaling
transform: encoder
transform: classifier
(122738, 33)
transform: missing
transform: outliers
transform: feature selection
transform: feature scaling
transform: encoder
transform: classifier

Model Accuracy: 0.7499382164515331
F1 Score (Macro): 0.45442920459334346


## B. Naive Bayes

In [48]:
reset_import()

from sklearn.naive_bayes import GaussianNB

import warnings

warnings.filterwarnings('ignore')

reset_import()

from src.compose import CustomPipeline

pipeline = CustomPipeline(args=[
    ('missing', MissingValueHandler(strategy=CustomTransformer(transformers=[
        (SimpleImputer(strategy='mean'), numerical_features),
        (SimpleImputer(strategy='most_frequent'), categorical_features)
    ]))),
    ('duplicate', DuplicateHandler(strategy=DropDuplicateHandlerStrategy())),
    ('outliers', OutlierHandler(strategy=CustomTransformer(transformers=[
        (BoundOutlierHandlerStrategy(), numerical_features)
    ]))),
    ('feature selection', FeatureSelectionHandler(strategy=CustomTransformer(transformers=[
        (RandomForestFeatureSelectionHandlerStrategy(), numerical_features),
    ]))),
    ('feature scaling', ScaleHandler(strategy=ZScoreScaleHandlerStrategy()), 'numerical_features'),
    ('encoder', PipelineComponent(strategy=None).set_io_strategy(
        LabelEncoderIOTransformerStrategy(strategy=CustomTransformer(transformers=[
            (SKLearnLabelEncoderProxy(), categorical_features),
        ])),
    )),
    ('classifier', PipelineComponent(strategy=None).set_io_strategy(
        ClassifierTransformerStrategy(strategy=Proxy(GaussianNB())))),
], pl_property=PipelineProperty(
    {'numerical_features': numerical_features, 'categorical_features': categorical_features}), verbose=True,
    only_once=['duplicate'])

X_train_transformed, _ = pipeline.fit_transform(X_train, y_train)

check_for_completeness(X_train_transformed)

_, y_pred = pipeline.transform(X_val)

accuracy = accuracy_score(y_val, y_pred)
print("\nModel Accuracy:", accuracy)

f1_macro = f1_score(y_val, y_pred, average='macro')
print(f"F1 Score (Macro): {f1_macro}")

fit: missing
fit: duplicate
fit: outliers
fit: feature selection
fit: feature scaling
fit: encoder
fit: classifier
transform: missing
transform: outliers
transform: feature selection
transform: feature scaling
transform: encoder
transform: classifier
(122738, 33)
transform: missing
transform: outliers
transform: feature selection
transform: feature scaling
transform: encoder
transform: classifier

Model Accuracy: 0.5347603748835618
F1 Score (Macro): 0.2710167382577316


## C. ID3

In [59]:
reset_import()

import warnings

warnings.filterwarnings('ignore')

reset_import()

from src.compose import CustomPipeline

pipeline = CustomPipeline(args=[
    ('missing', MissingValueHandler(strategy=CustomTransformer(transformers=[
        (SimpleImputer(strategy='mean'), numerical_features),
        (SimpleImputer(strategy='most_frequent'), categorical_features)
    ]))),
    ('duplicate', DuplicateHandler(strategy=DropDuplicateHandlerStrategy())),
    ('outliers', OutlierHandler(strategy=CustomTransformer(transformers=[
        (BoundOutlierHandlerStrategy(), numerical_features)
    ]))),
    ('feature selection', FeatureSelectionHandler(strategy=CustomTransformer(transformers=[
        (RandomForestFeatureSelectionHandlerStrategy(), numerical_features),
    ]))),
    ('feature scaling', ScaleHandler(strategy=ZScoreScaleHandlerStrategy()), 'numerical_features'),
    ('encoder', PipelineComponent(strategy=None).set_io_strategy(
        LabelEncoderIOTransformerStrategy(strategy=CustomTransformer(transformers=[
            (SKLearnLabelEncoderProxy(), categorical_features),
        ])),
    )),
    ('classifier', PipelineComponent(strategy=None).set_io_strategy(
        ClassifierTransformerStrategy(strategy=Proxy(DecisionTreeClassifier(criterion='entropy', random_state=0))))),
], pl_property=PipelineProperty(
    {'numerical_features': numerical_features, 'categorical_features': categorical_features}), verbose=True,
    only_once=['duplicate'])

X_train_transformed, _ = pipeline.fit_transform(X_train, y_train)

check_for_completeness(X_train_transformed)

_, y_pred = pipeline.transform(X_val)

accuracy = accuracy_score(y_val, y_pred)
print("\nModel Accuracy:", accuracy)

f1_macro = f1_score(y_val, y_pred, average='macro')
print(f"F1 Score (Macro): {f1_macro}")

fit: missing
fit: duplicate
fit: outliers
fit: feature selection
fit: feature scaling
fit: encoder
fit: classifier
transform: missing
transform: outliers
transform: feature selection
transform: feature scaling
transform: encoder
transform: classifier
(122738, 33)
transform: missing
transform: outliers
transform: feature selection
transform: feature scaling
transform: encoder
transform: classifier

Model Accuracy: 0.7767807919700397
F1 Score (Macro): 0.5406304285963492


## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`